1. Title: Car Evaluation Database

2. Sources:
   (a) Creator: Marko Bohanec
   (b) Donors: Marko Bohanec   (marko.bohanec@ijs.si)
               Blaz Zupan      (blaz.zupan@ijs.si)
   (c) Date: June, 1997

3. Past Usage:

   The hierarchical decision model, from which this dataset is
   derived, was first presented in 

   M. Bohanec and V. Rajkovic: Knowledge acquisition and explanation for
   multi-attribute decision making. In 8th Intl Workshop on Expert
   Systems and their Applications, Avignon, France. pages 59-78, 1988.

   Within machine-learning, this dataset was used for the evaluation
   of HINT (Hierarchy INduction Tool), which was proved to be able to
   completely reconstruct the original hierarchical model. This,
   together with a comparison with C4.5, is presented in

   B. Zupan, M. Bohanec, I. Bratko, J. Demsar: Machine learning by
   function decomposition. ICML-97, Nashville, TN. 1997 (to appear)

4. Relevant Information Paragraph:

   Car Evaluation Database was derived from a simple hierarchical
   decision model originally developed for the demonstration of DEX
   (M. Bohanec, V. Rajkovic: Expert system for decision
   making. Sistemica 1(1), pp. 145-157, 1990.). The model evaluates
   cars according to the following concept structure:

   CAR                      car acceptability
   . PRICE                  overall price
   . . buying               buying price
   . . maint                price of the maintenance
   . TECH                   technical characteristics
   . . COMFORT              comfort
   . . . doors              number of doors
   . . . persons            capacity in terms of persons to carry
   . . . lug_boot           the size of luggage boot
   . . safety               estimated safety of the car

   Input attributes are printed in lowercase. Besides the target
   concept (CAR), the model includes three intermediate concepts:
   PRICE, TECH, COMFORT. Every concept is in the original model
   related to its lower level descendants by a set of examples (for
   these examples sets see http://www-ai.ijs.si/BlazZupan/car.html).

   The Car Evaluation Database contains examples with the structural
   information removed, i.e., directly relates CAR to the six input
   attributes: buying, maint, doors, persons, lug_boot, safety.

   Because of known underlying concept structure, this database may be
   particularly useful for testing constructive induction and
   structure discovery methods.

5. Number of Instances: 1728
   (instances completely cover the attribute space)

6. Number of Attributes: 6

7. Attribute Values:

   buying       v-high, high, med, low
   maint        v-high, high, med, low
   doors        2, 3, 4, 5-more
   persons      2, 4, more
   lug_boot     small, med, big
   safety       low, med, high

8. Missing Attribute Values: none

9. Class Distribution (number of instances per class)

| class | N | N[%] | 
| -: | :-: | :-: | 
| unacc | 1210  |   (70.023 %) 
|    acc    |    384  |   (22.222 %) 
|   good    |    69  |   ( 3.993 %) 
|   v-good   |   65   |  ( 3.762 %) 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_recall_fscore_support

In [2]:
column_headers = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'y']
data = pd.read_csv('./car.data', names=column_headers, index_col=False)
data.head(5)

,buying,maint,doors,persons,lug_boot,safety,y
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
data.shape

(1728, 7)

## Exploratory Data Analysis

In [4]:
# No null values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   y         1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [5]:
# Identify the categorical variables
categorical_variables = [col for col in data.columns if data[col].dtype=='object']

# Inspect categorical variables
data[categorical_variables].describe()

,buying,maint,doors,persons,lug_boot,safety,y
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,med,med,3,more,big,med,unacc
freq,432,432,432,576,576,576,1210


In [6]:
for column in column_headers:
    print(round(data[column].value_counts()/1728*100,3))

med      25.0
low      25.0
high     25.0
vhigh    25.0
Name: buying, dtype: float64
med      25.0
low      25.0
high     25.0
vhigh    25.0
Name: maint, dtype: float64
3        25.0
4        25.0
2        25.0
5more    25.0
Name: doors, dtype: float64
more    33.333
4       33.333
2       33.333
Name: persons, dtype: float64
big      33.333
small    33.333
med      33.333
Name: lug_boot, dtype: float64
med     33.333
low     33.333
high    33.333
Name: safety, dtype: float64
unacc    70.023
acc      22.222
good      3.993
vgood     3.762
Name: y, dtype: float64


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   y         1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [8]:
column_headers = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'y']
data = pd.read_csv('./car.data', names=column_headers, index_col=False)

In [9]:
buying_mapper = {'low':1, 'med':2, 'high':3, 'vhigh':4}
maint_mapper = {'low':1, 'med':2, 'high':3, 'vhigh':4}
doors_mapper = {'2':1, '3':2, '4':3, '5more':4}
persons_mapper = {'2':1, '4':2, 'more':3}
lug_boot = {'small':1, 'med':2, 'big':3}
safety_mapper = {'low':1, 'med':2, 'high':3}
y_mapper = {'unacc':1, 'acc':2, 'good':3, 'vgood':4}

In [10]:
data['buying'] = data['buying'].map(buying_mapper)
data['maint'] = data['maint'].map(maint_mapper)
data['doors'] = data['doors'].map(doors_mapper)
data['persons'] = data['persons'].map(persons_mapper)
data['lug_boot'] = data['lug_boot'].map(lug_boot)
data['safety'] = data['safety'].map(safety_mapper)
# data['y'] = data['y'].map(y_mapper)

In [11]:
# for col in column_headers:
#     data[col]=data[col].astype('Int64')

In [12]:
# No null values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   int64 
 1   maint     1728 non-null   int64 
 2   doors     1728 non-null   int64 
 3   persons   1728 non-null   int64 
 4   lug_boot  1728 non-null   int64 
 5   safety    1728 non-null   int64 
 6   y         1728 non-null   object
dtypes: int64(6), object(1)
memory usage: 94.6+ KB


In [13]:
data.head(5)

,buying,maint,doors,persons,lug_boot,safety,y
0,4,4,1,1,1,1,unacc
1,4,4,1,1,1,2,unacc
2,4,4,1,1,1,3,unacc
3,4,4,1,1,2,1,unacc
4,4,4,1,1,2,2,unacc


In [14]:
X = data[[x for x in data.columns if x != 'y']]
y = data['y']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)


In [22]:
# Adaboost

In [23]:
# define hyperparameters for GridSearch and model tuning
estimators = [
#     DecisionTreeClassifier(criterion='gini', max_depth=2, random_state=123), 
#     DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123), 
    DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=123), 
#     DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=123), 
    DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123),
    DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=123), 
              ]

hyperparameters = {'base_estimator':estimators, 'n_estimators': [50, 100, 150, 200], 'learning_rate':[0.5, 1, 1.5]}

# define evaluation procedure
adaboost = GridSearchCV(AdaBoostClassifier(random_state=123), hyperparameters, cv=5, scoring="accuracy", n_jobs=-1)
adaboost.fit(X_train, y_train.values.ravel())
print(f'Best Accuracy score: {adaboost.best_score_:.3f} with parameters: {adaboost.best_params_}')

Best Accuracy score: 0.958 with parameters: {'base_estimator': DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=123), 'learning_rate': 1.5, 'n_estimators': 150}


In [24]:
adaboost_prediction = adaboost.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, adaboost_prediction)*100:.1f}')
print(f'F1 Score: {f1_score(y_test, adaboost_prediction, average="weighted")*100:.1f}')

Accuracy: 94.0
F1 Score: 94.2


In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test, adaboost_prediction))

              precision    recall  f1-score   support

         acc       0.80      0.97      0.88       115
        good       1.00      0.90      0.95        21
       unacc       0.99      0.93      0.96       363
       vgood       1.00      0.95      0.97        20

    accuracy                           0.94       519
   macro avg       0.95      0.94      0.94       519
weighted avg       0.95      0.94      0.94       519



## SVM 

In [26]:
# Try linear only (runtime too long)
hyperparameters = {'kernel': ['linear', 'rbf', 'poly'], 
                   'C':[0.2, 1, 5], 
#                    'tol':[0.5, 0.1, 0.05],
                   'decision_function_shape':['ovr'],
                   'gamma':[1e-1]
                  }

# define evaluation procedure
svm = GridSearchCV(SVC(random_state=123), hyperparameters, scoring="accuracy", cv=5, n_jobs=-1)
svm.fit(X_train, y_train.values.ravel())
print(f'Best F1 score: {svm.best_score_:.3f} with parameters: {svm.best_params_}')

Best F1 score: 0.970 with parameters: {'C': 5, 'decision_function_shape': 'ovr', 'gamma': 0.1, 'kernel': 'rbf'}


In [27]:
svm_prediction = svm.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, svm_prediction)*100:.1f}')
print(f'F1 Score: {f1_score(y_test, svm_prediction, average="weighted")*100:.1f}')

Accuracy: 96.7
F1 Score: 96.7


In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_test, svm_prediction))

              precision    recall  f1-score   support

         acc       0.91      0.96      0.93       115
        good       0.95      0.90      0.93        21
       unacc       0.99      0.97      0.98       363
       vgood       0.95      1.00      0.98        20

    accuracy                           0.97       519
   macro avg       0.95      0.96      0.95       519
weighted avg       0.97      0.97      0.97       519



# Train Best Model on whole dataset 

In [29]:
clf = SVC(random_state=123, **svm.best_params_)
clf.fit(X, y.values.ravel())

SVC(C=5, gamma=0.1, random_state=4222)

## Predict test data

In [33]:
clf.plot_tree(clf_gini.fit(X_train, y_train))

AttributeError: 'SVC' object has no attribute 'plot_tree'